In [2]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split
import xgboost as xgb

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_df = pd.read_csv('/content/drive/Shared drives/#PA_Customer Revenue Prediction/input_train_v3.csv', sep = ',')

In [0]:
test_df = pd.read_csv('/content/drive/Shared drives/#PA_Customer Revenue Prediction/input_test_v3.csv', sep = ',')

In [0]:
input_test = pd.read_csv('/content/drive/Shared drives/#PA_Customer Revenue Prediction/input_test_v3.csv', sep = ',')

In [0]:
test_df = input_test


In [0]:
visit_id = input_test['fullVisitorId_str']

In [0]:
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot
import pandas as pd


In [0]:
train_df = train_df.iloc[:, 1:]
train_df = train_df.fillna(0)

In [0]:
test_df = test_df.iloc[:, 1:]

In [0]:
traindf, testdf = train_test_split(train_df, test_size = 0.15, random_state = 20)

In [0]:
# whether return or not?
x_train = traindf.iloc[:,:37]
y_train_return = traindf['will_return']
y_train_value = traindf["how_much"]

x_test = testdf.iloc[:, :37]
y_test_return = testdf['will_return']
y_test_value = testdf["how_much"]

In [47]:
x_train.head()

,"('channelGrouping', 'max')","('visitNumber', 'max')","('visitNumber', 'count')","('device_browser', 'max')","('device_operatingSystem', 'max')","('deviceCategory', 'max')","('geoNetwork_continent', 'max')","('geoNetwork_subContinent', 'max')","('geoNetwork_country', 'max')","('geoNetwork_region', 'max')","('geoNetwork_metro', 'max')","('geoNetwork_city', 'max')","('geoNetwork_networkDomain', 'max')","('trafficSource_source', 'max')","('trafficSource_medium', 'max')","('trafficSource_adwordsClickInfo.isVideoAd', 'mean')","('device_isMobile', 'mean')","('trafficSource_isTrueDirect', 'mean')","('totals_bounces', 'sum')","('totals_hits', 'sum')","('totals_hits', 'mean')","('totals_hits', 'min')","('totals_hits', 'max')","('totals_hits', 'median')","('totals_hits', 'std')","('totals_pageviews', 'sum')","('totals_pageviews', 'mean')","('totals_pageviews', 'min')","('totals_pageviews', 'max')","('totals_pageviews', 'median')","('totals_pageviews', 'std')","('totals_transactionRevenue', 'sum')","('totals_transactions', 'sum')","('first_ses_from_the_period_start', '')","('last_ses_from_the_period_end', '')","('interval_dates', '')",unique_date_num
671330,4,1,1,35,20,0,4,13,217,131,60,458,38725,99,5,1.0,0.0,1.0,0.0,3,3.000000,3,3,3.0,0.000000,2.0,2.000000,2.0,2.0,2.0,0.000000,0.0,0.0,375.0,262.0,0.0,1
1031924,3,1,1,35,23,1,2,12,218,73,100,540,0,99,4,1.0,1.0,1.0,0.0,4,4.000000,4,4,4.0,0.000000,4.0,4.000000,4.0,4.0,4.0,0.000000,0.0,0.0,409.0,228.0,0.0,1
504914,6,1,1,72,23,2,2,12,218,73,100,94,31811,300,6,1.0,1.0,1.0,0.0,12,12.000000,12,12,12.0,0.000000,9.0,9.000000,9.0,9.0,9.0,0.000000,0.0,0.0,193.0,444.0,0.0,1
704694,3,1,1,74,1,1,2,12,218,149,4,52,0,99,3,0.0,1.0,1.0,1.0,1,1.000000,1,1,1.0,0.000000,1.0,1.000000,1.0,1.0,1.0,0.000000,0.0,0.0,432.0,205.0,0.0,1
301010,2,2,3,35,6,0,2,12,218,73,100,540,0,0,0,1.0,0.0,0.0,0.0,44,14.666667,2,25,17.0,11.676187,37.0,12.333333,2.0,20.0,15.0,9.291573,0.0,0.0,31.0,551.0,55.0,3


In [0]:
xgb_r = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
xgb_grid = RandomizedSearchCV(xgb_r,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(x_train,
         y_train_value)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  18 out of  18 | elapsed: 116.1min finished
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


0.006701823721211836
{'subsample': 0.7, 'silent': 1, 'objective': 'reg:linear', 'nthread': 4, 'n_estimators': 500, 'min_child_weight': 4, 'max_depth': 5, 'learning_rate': 0.03, 'colsample_bytree': 0.7}


In [0]:
# final regression model
xgb_r = XGBRegressor(subsample =  0.7, silent = 1, objective = 'reg:linear', nthread = 4, n_estimators = 500, min_child_weight = 4, max_depth = 5, learning_rate = 0.03, colsample_bytree =  0.7)

In [49]:
xgb_r.fit(x_train, y_train_value)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.03, max_delta_step=0,
             max_depth=5, min_child_weight=4, missing=None, n_estimators=500,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
             subsample=0.7, verbosity=1)

In [0]:
# predict revenue based on validation set
predict_revenue = xgb_r.predict(x_test)

In [0]:
# prediction on real test data
predict_revenue_final= xgb_r.predict(test_df)

In [0]:
predict_revenue_final = pd.DataFrame(predict_revenue_final)

In [53]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test_value, predict_revenue)

0.11697325301397242

In [55]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test_value, predict_revenue)

0.013101233577934614

In [0]:
test_df_final = pd.read_csv('/content/drive/Shared drives/#PA_Customer Revenue Prediction/input_test_v3.csv', sep = ',')

In [57]:
test_df_final.columns.values

array(['fullVisitorId_str', "('channelGrouping', 'max')",
       "('visitNumber', 'max')", "('visitNumber', 'count')",
       "('device_browser', 'max')", "('device_operatingSystem', 'max')",
       "('deviceCategory', 'max')", "('geoNetwork_continent', 'max')",
       "('geoNetwork_subContinent', 'max')",
       "('geoNetwork_country', 'max')", "('geoNetwork_region', 'max')",
       "('geoNetwork_metro', 'max')", "('geoNetwork_city', 'max')",
       "('geoNetwork_networkDomain', 'max')",
       "('trafficSource_source', 'max')",
       "('trafficSource_medium', 'max')",
       "('trafficSource_adwordsClickInfo.isVideoAd', 'mean')",
       "('device_isMobile', 'mean')",
       "('trafficSource_isTrueDirect', 'mean')",
       "('totals_bounces', 'sum')", "('totals_hits', 'sum')",
       "('totals_hits', 'mean')", "('totals_hits', 'min')",
       "('totals_hits', 'max')", "('totals_hits', 'median')",
       "('totals_hits', 'std')", "('totals_pageviews', 'sum')",
       "('totals_pagevie

In [59]:
fullVisitId = pd.DataFrame(test_df_final['fullVisitorId_str'])
fullVisitId

,fullVisitorId_str
0,0000018966949534117a
1,0000039738481224681a
2,0000073585230191399a
3,0000087588448856385a
4,0000149787903119437a
...,...
296525,9999862054614696520a
296526,9999898168621645223a
296527,999990167740728398a
296528,9999915620249883537a


In [0]:
#
sub_final = fullVisitId.merge(predict_revenue_final, left_index=True, right_index=True)

In [0]:
#
sub_final = sub_final.rename(columns={0 : 'PredictedLogRevenue'})

In [0]:
sub_final = sub_final[['fullVisitorId_str', 'PredictedLogRevenue']]

In [65]:
super_final = pd.read_csv('/content/drive/Shared drives/#PA_Customer Revenue Prediction/tst4.csv')
super_final['PredictedLogRevenue'] = sub_final['PredictedLogRevenue']

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
super_final

,fullVisitorId,PredictedLogRevenue
0,0000018966949534117,0.007360
1,0000039738481224681,0.008947
2,0000073585230191399,0.006927
3,0000087588448856385,0.006479
4,0000149787903119437,0.006595
...,...,...
296525,9999862054614696520,0.000341
296526,9999898168621645223,0.006594
296527,999990167740728398,0.008499
296528,9999915620249883537,0.007244


In [0]:
super_final.to_csv('/content/drive/Shared drives/#PA_Customer Revenue Prediction/final_submission.csv', index = None, header = True)